In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - fit_GE_mass_mock.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Fit density profiles to mock APOGEE data
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, pdb, time, tqdm, warnings, multiprocessing, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib
import matplotlib.pyplot as plt
import corner

# mwdust, isodist stuff, galpy stuff
import mwdust
from isodist import FEH2Z, Z2FEH
from galpy.util import coords

# Fitting, optimization and statistics
import emcee
import scipy.optimize

# Project specific
sys.path.insert(0,'../../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import iso as piso
from ges_mass import util as putil
from ges_mass import plot as pplot

### Scale parameters
ro = 8.275
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup

%matplotlib inline
plt.style.use('../../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries
Keywords, pathing, loading and setup

In [ ]:
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'NPROCS']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,nprocs\
    = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]

In [ ]:
## Pathing
data_dir = base_dir+'data/'
version_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
ga_dir = data_dir+'gaia_apogee/'+version_dir
gap_dir = data_dir+'gaia_apogee_processed/'+version_dir
ksf_dir = data_dir+'ksf/'+version_dir

In [ ]:
## Filenames
apogee_SF_filename = ga_dir+'apogee_SF.dat'
apogee_effSF_filename = ga_dir+\
    'apogee_effSF_grid_inclArea_z0.001_logAge10.0.dat'
apogee_effSF_mask_filename = ga_dir+\
    'apogee_effSF_grid_mask_z0.001_logAge10.0.npy'
iso_grid_filename = ga_dir+'iso_grid.npy'

In [ ]:
## Loading
# selection function stuff
with open(apogee_SF_filename, 'rb') as f:
    print('\nLoading APOGEE sel. func. from '+apogee_SF_filename)
    apogee_SF = pickle.load(f)
with open(apogee_effSF_filename,'rb') as f:
    print('\nLoading APOGEE eff. sel. func. from '+apogee_effSF_filename)
    apogee_effSF_grid_inclArea = pickle.load(f)
print('\nLoading APOGEE eff. sel. func. mask from '+apogee_effSF_mask_filename)
apogee_effSF_mask = np.load(apogee_effSF_mask_filename)

# dust map
dmap = mwdust.Combined19(filter='2MASS H') # dustmap from mwdust, use most recent

# isochrone
print('\nLoading isochrone grid from '+iso_grid_filename)
iso_grid = np.load(iso_grid_filename)
z = 0.0010 # Normal parameters for single isochrone
log_age = 10.0
iso = iso_grid[(iso_grid['Zini']==z) & (iso_grid['logAge']==log_age)]
iso = iso[iso['logL'] > -9]

In [ ]:
## Effective selection function grid
# Distance modulus grid
dmods,ds = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=ro,zo=zo)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

# Include the distance modulus Jacobian
Jac_dmod = ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])
Jac_rad = (np.pi/180.)**2.
apof = apof * Jac_dmod * Jac_rad

# JKmins
jkmins = np.array([apogee_SF.JKmin(apogee_SF._locations[i]) \
                   for i in range(len(apogee_SF._locations))])
jkmins = jkmins[apogee_effSF_mask]

In [ ]:
def load_mock_data(mock_number):
    # Filenames
    mock_path = '../data/mock_'+mock_number+'/'
    allstar_filename = mock_path+'/allstar.npy'
    orbs_filename = mock_path+'/orbs.pkl'
    data_omask_filename = mock_path+'/omask.npy'

    # Load data
    with open(mock_path+'orbs.pkl','rb') as f:
        orbs_nomask = pickle.load(f)
    allstar_nomask = np.load(allstar_filename)
    data_mask = np.load(data_omask_filename)

    # Setup data array and mask the data
    orbs = orbs_nomask[data_mask]
    allstar = allstar_nomask[data_mask]
    mrpz = np.array([orbs.R(use_physical=True).value,
                     orbs.phi(use_physical=True).value,
                     orbs.z(use_physical=True).value]).T
    return orbs,allstar,mrpz,data_mask

### Load mock data

In [ ]:
# Filenames
mock_number = '40'
mock_path = '../data/mock_'+mock_number+'/'
allstar_filename = mock_path+'/allstar.npy'
orbs_filename = mock_path+'/orbs.pkl'
data_omask_filename = mock_path+'/omask.npy'

# Load
with open(mock_path+'orbs.pkl','rb') as f:
    orbs_nomask = pickle.load(f)
allstar_nomask = np.load(allstar_filename)
data_mask = np.load(data_omask_filename)

# Setup data array and mask the data
orbs = orbs_nomask[data_mask]
allstar = allstar_nomask[data_mask]
mrpz = np.array([orbs.R(use_physical=True).value,
                 orbs.phi(use_physical=True).value,
                 orbs.z(use_physical=True).value]).T

### Do the fitting

In [ ]:
# Definitions
densfunc = pdens.triaxial_single_angle_zvecpa
# alpha, p, q, eta, theta, pa
init = np.array([2.0, 0.5, 0.5, 0.5, 0.5, 0.5])
ndim = len(init)
nwalkers = 200
nit = 200
ncut = 100
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apof)

# Maximum likelihood
opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
    Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
    full_output=True)
print(opt[0])

#pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]

with multiprocessing.Pool(nprocs) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
        args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
        pool=pool)
    print('Generating MCMC samples...')
    for i, result in enumerate(sampler.sample(pos, iterations=nit)):
        if (i+1)%10 == 0: print('sampled '+str(i+1)+'/'+str(nit))
        continue
    # Flatten the ensemble of walkers to a set of samples, remove ncut from each
    # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
    samples = sampler.get_chain(flat=True,discard=ncut)

In [ ]:
labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=[3.5,0.5,1.,0.,1.,1./6.], truth_color='Red', 
                    labels=labels)
fig.show()

## Calculate the mass

In [ ]:
# Parameters
nfield = apof.shape[0]
n_mass = 100
assert n_mass < samples.shape[0]
mass_int_type = 'spherical'
r_min,r_max = 0.1, 70.

### Isochrone mass factors

In [ ]:
# Determine the isochrone mass fraction factors for each field 
isofactors = np.zeros(nfield)
for i in range(nfield):
    # Calculate the full isochrone mask for the average mass calculation
    avmass_isomask = (Z2FEH(iso['Zini']) > feh_range[0]) &\
                     (Z2FEH(iso['Zini']) < feh_range[1]) &\
                     (iso['Jmag']-iso['Ksmag'] > jkmins[i]) &\
                     (iso['logg'] > logg_range[0]) &\
                     (iso['logg'] < logg_range[1]) &\
                     (iso['logAge'] >= 10)
    avmass = piso.average_mass(iso[avmass_isomask])
    # Calculate a broader isochrone mask for the mass ratio calculation
    massratio_isomask = (Z2FEH(iso['Zini']) > feh_range[0]) &\
                        (Z2FEH(iso['Zini']) < feh_range[1]) &\
                        (iso['logAge'] >= 10)
    massratio = piso.mass_ratio(iso[massratio_isomask], logg_range=logg_range,
                                jk_range=[jkmins[i],999.])
    isofactors[i] = avmass/massratio

In [ ]:
print(avmass)
print(np.unique(isofactors))

In [ ]:
massratio

### Something is wrong here... The isochrone factors should be smaller?

In [ ]:
print(avmass)
print(np.unique(isofactors))

In [ ]:
# Determine the isochrone mass fraction factors for each field 
isofactors = np.zeros(nfield)
for i in range(nfield):
    # Calculate the full isochrone mask for the average mass calculation
    avmass_isomask = (Z2FEH(iso['Zini']) > feh_range[0]) &\
                     (Z2FEH(iso['Zini']) < feh_range[1]) &\
                     (iso['Jmag']-iso['Ksmag'] > jkmins[i]) &\
                     (iso['logg'] > logg_range[0]) &\
                     (iso['logg'] < logg_range[1]) &\
                     (iso['logAge'] >= 10) &\
                     (iso['logL'] > -9)
    avmass = piso.average_mass(iso[avmass_isomask])
    # Calculate a broader isochrone mask for the mass ratio calculation
    massratio_isomask = (Z2FEH(iso['Zini']) > feh_range[0]) &\
                        (Z2FEH(iso['Zini']) < feh_range[1]) &\
                        (iso['logAge'] >= 10) &\
                        (iso['logL'] > -9)
    massratio = piso.mass_ratio(iso[massratio_isomask], logg_range=logg_range,
                                jk_range=[jkmins[i],999.])
    isofactors[i] = avmass/massratio

In [ ]:
print(avmass)
print(np.unique(isofactors))

In [ ]:
# Set up grid for integration
if mass_int_type == 'spherical':
    rthetaphigrid = np.mgrid[r_min:r_max:150j,0:np.pi:150j,0:2*np.pi:150j]
    dr = (r_max-r_min)/149
    dtheta = (np.pi-0.)/149
    dphi = (2*np.pi-0.)/149
    shape = np.shape(rthetaphigrid.T)
    rthetaphigrid = rthetaphigrid.T.reshape(np.product(shape[:3]),shape[3])
    deltafactor = rthetaphigrid[:,0]**2*np.sin(rthetaphigrid[:,1])*dr*dtheta*dphi
    x = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.cos(rthetaphigrid[:,2])
    y = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.sin(rthetaphigrid[:,2])
    z = rthetaphigrid[:,0]*np.cos(rthetaphigrid[:,1])
    xyzgrid = np.dstack([x,y,z])[0]
    rphizgrid = coords.rect_to_cyl(xyzgrid[:,0], xyzgrid[:,1], xyzgrid[:,2])
    rphizgrid = np.dstack([rphizgrid[0],rphizgrid[1],rphizgrid[2]])[0]
# if mass_int_type == 'cartesian':
#     xyzgrid = np.mgrid[-50.:50.:150j,-50.:50.:150j,-50.:50.:150j]
#     delta = xyzgrid[0,:,0,0][1]-xyzgrid[0,:,0,0][0]
#     deltafactor = delta**3
#     shape = np.shape(xyzgrid.T)
#     xyzgrid = xyzgrid.T.reshape(np.product(shape[:3]),shape[3])
#     rphizgrid = coords.rect_to_cyl(xyzgrid[:,0], xyzgrid[:,1], xyzgrid[:,2])
#     rphizgrid = np.dstack([rphizgrid[0],rphizgrid[1],rphizgrid[2]])[0]
if densfunc is pdens.triaxial_single_angle_zvecpa_plusexpdisk:# or \
   #densfunc is pdens.triaxial_einasto_zvecpa_plusexpdisk or \
   #densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk or \
   #densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk:
    masses = np.zeros((n_mass,3))
else:
    masses = np.zeros(n_mass)
facs = np.zeros(n_mass)

In [ ]:
# Calculate the mass
sample_randind = np.random.choice(len(samples),n_mass,replace=False)
for ii,params in enumerate(samples[sample_randind]):
    #params = [3.5,0.5,1.,0.,1.,1./6.]
    rate = densfunc(Rgrid,phigrid,zgrid,params=params)*effsel
    sumrate = np.sum(rate.T/isofactors)
    fac = n_star/sumrate
    # fac = norm*(180./np.pi)**2 # Get rid of this normalization factor??
    if mass_analytic:
        #only for spherical power law!
        rsun = np.sqrt(ro**2+zo**2)
        alpha = params[0]
        integral = 4*np.pi*((rsun**alpha*r_max**(3-alpha))/(3-alpha)-(rsun**alpha*r_min**(3-alpha))/(3-alpha))
        masses[ii] = integral*fac
    else:
        if densfunc is pdens.triaxial_single_angle_zvecpa_plusexpdisk: # or \
           # densfunc is pdens.triaxial_einasto_zvecpa_plusexpdisk or \
           # densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk or \
           # densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk:
            denstxyz = densfunc(rphizgrid[:,0],rphizgrid[:,1],rphizgrid[:,2], 
                                params=params, split=True)
            halodens = denstxyz[0]*fac
            diskdens = denstxyz[1]*fac
            fulldens = densfunc(rphizgrid[:,0],rphizgrid[:,1],rphizgrid[:,2], params=params)*fac
            masses[ii] = np.sum(halodens*deltafactor), np.sum(diskdens*deltafactor), np.sum(fulldens*deltafactor)
        else:
            denstxyz = densfunc(rphizgrid[:,0],rphizgrid[:,1],rphizgrid[:,2], params=params)*fac
            masses[ii] =  np.sum(denstxyz*deltafactor)
    #densfunc = lambda r,phi,z: r*densfunc(r,phi,z,params=params)
    #integral = nquad(densfunc, [[3.,50.],[0.,2*np.pi],[-30.,30.]])
    #masses[ii] = integral[0]*fac
    facs[ii] = fac

In [ ]:
np.median(masses)/1e8/np.pi

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.hist(masses, bins=20, histtype='step')
ax.set_xlabel('Mass')
ax.set_ylabel('N')

fig.show()

In [ ]:
5e8/np.median(masses)

## Do this for a number of spherical power laws to check numerical accuracy

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [3.5,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('50')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [3.5,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('51')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [3.5,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('52')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.5,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('53')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)
print(n_star)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.5,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('54')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)
print(n_star)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.5,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('55')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)
print(n_star)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.0,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('56')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.0,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('57')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.0,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('58')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.0,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('59')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.0,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('60')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

In [ ]:
# Definitions
densfunc = pdens.spherical
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.0,]
init = np.array([2.0,])
ndim = len(init)
nwalkers, nit, ncut = 25, 200, 100
n_mass, mass_int_type, mass_analytic = 100, 'spherical', False
r_min,r_max = 1.0, 70.
nfield = apof.shape[0]

# Files
orbs,allstar,mrpz,_ = load_mock_data('61')
Rdata,phidata,zdata = mrpz.T
n_star = len(orbs)

t1 = time.time()
# Fit the data
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=[3.5,], nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)
t2 = time.time()
print('MCMC took '+str(t2-t1)+'s')

# Corner plot
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

# Calculate mass forced to be the correct parameters
samples_force = np.atleast_2d(np.array([mcmc_truths[0],mcmc_truths[0]])).T
mass_force, _ = pmass.mass_from_density_samples(samples=samples_force, 
    densfunc=densfunc, n_star=len(orbs), effsel=apof, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=1, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    truths=[5,], truth_color='Red', labels=['mass'])
fig.show()

print('Mass: '+str(np.median(masses)/1e8)+' e8')
print('Forced mass: '+str(np.median(mass_force)/1e8)+' e8')

### Try some triaxial power laws, just the mass

In [ ]:
from galpy import potential
from astropy import units as apu

In [ ]:
pot = potential.TwoPowerTriaxialPotential(a=1,b=0.5,c=1.,alpha=3.5,beta=3.5,pa=np.pi/6*apu.rad)

In [ ]:
mock_numbers = ['40','41','42','43','44','45']

# [alpha,b,c,theta,eta,pa]
# theta normalized 0 to 2pi
# eta normalized -1 to 1
# pa normalized 0 to pi
params_correct = [
    [3.5, 0.5, 1.0, 0.0      , 1.0  , np.pi/6],
    [3.5, 0.5, 0.8, 0.0      , 1.0  , np.pi/5],
    [3.5, 1.0, 0.5, 0.0      , 1.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, np.pi/2. , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.707, np.pi/6]
]

r_min = 0.1 
r_max = 70. 

densfunc = pdens.triaxial_single_angle_zvecpa

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for k in range(len(mock_numbers)):
        
        #if k > 2: continue
        
        # Files
        orbs,allstar,mrpz,_ = load_mock_data(mock_numbers[k])
        Rdata,phidata,zdata = mrpz.T
        n_star = len(orbs)
        
        # Calculate mass
        samples = np.array([params_correct[k]])
        masses, facs = pmass.mass_from_density_samples(samples=samples, 
            densfunc=densfunc, n_star=len(orbs), effsel=apof, 
            effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
            logg_range=logg_range, jkmins=jkmins, n_mass=1, 
            mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, 
            zo=zo)
        print('Mass: '+str(np.median(masses)/1e8)+'e8')

## Confirm that the different forms of mass calculation agree for spherical power law
So seems like for low `r_min` (less than 1 kpc) and power laws steeper than `alpha=3` that having a fine grid for spherical integration is important

In [ ]:
r_min = 1.
r_max = 70.
alpha = 3.5
densfunc = pdens.spherical

In [ ]:
# Power law equation
rsun = np.sqrt(ro**2+zo**2)
integral = 4*np.pi*rsun**alpha*((r_max**(3-alpha))/(3-alpha)-\
                                (r_min**(3-alpha))/(3-alpha))
print('Power law equation: '+str(integral))

In [ ]:
def spherical_integration_grid(r_min,r_max,n_edge_r,n_edge_theta,n_edge_phi):
    '''spherical_integration_grid:
    
    Make a spherical integration grid. For shallow power laws (alpha<3) 
    n_edge_r can be ~ 200. For steep power laws (alpha>3) and low r_min 
    (< 1kpc) consider n_edge_r ~ 500. Note the number of bins for each grid is 
    the number of edges - 1
    
    
    Args:
        r_min (float) - Minimum radius for the grid
        r_max (float) - Maximum radius for the grid
        n_edge_r (int) - Number of edges for the radial grid (n_bin = n_edge-1)
        n_edge_theta (int) - Number of edges for the theta grid (polar)
        n_edge_phi (int) - NUmber of edges for the phi grid (azimuthal)
    
    Returns:
        rphizgrid (array) - Array of shape (N,3) where N is nr*ntheta*nphi 
            of cylindrical R,phi,z coordinates for the grid
        delta (array) - Array of shape (N) where N is nr*ntheta*nphi of 
            integral delta factors: r^2 sin(theta) dr dtheta dphi
    '''
    rgrid = np.linspace(r_min,r_max,n_edge_r)
    dr = np.diff(rgrid)[0]
    rgrid += dr/2
    thetagrid = np.linspace(0,np.pi,n_edge_theta)
    dtheta = np.diff(thetagrid)[0]
    thetagrid += dtheta/2
    phigrid = np.linspace(0,2*np.pi,n_edge_phi)
    dphi = np.diff(phigrid)[0]
    phigrid += dphi/2
    rthetaphigrid = np.asarray(np.meshgrid(rgrid[:-1],thetagrid[:-1],phigrid[:-1]))
    nbin = (n_edge_r-1)*(n_edge_theta-1)*(n_edge_phi-1)
    rthetaphigrid = rthetaphigrid.reshape(3,nbin).T
    delta = rthetaphigrid[:,0]**2*np.sin(rthetaphigrid[:,1])*dr*dtheta*dphi
    x = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.cos(rthetaphigrid[:,2])
    y = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.sin(rthetaphigrid[:,2])
    z = rthetaphigrid[:,0]*np.cos(rthetaphigrid[:,1])
    xyzgrid = np.dstack([x,y,z])[0]
    Rphizgrid = coords.rect_to_cyl(xyzgrid[:,0], xyzgrid[:,1], xyzgrid[:,2])
    Rphizgrid = np.dstack([Rphizgrid[0],Rphizgrid[1],Rphizgrid[2]])[0]
    return Rphizgrid,delta

In [ ]:
Rphizgrid,delta = spherical_integration_grid(r_min,r_max,500,100,100)
denstxyz = densfunc(Rphizgrid[:,0],Rphizgrid[:,1],Rphizgrid[:,2], params=[alpha,])
integral =  np.sum(denstxyz*delta)
print('Spherical grid integration: '+str(integral))

In [ ]:
nedge = 500
rgrid = np.linspace(r_min,r_max,nedge)
dr = np.diff(rgrid)[0]
rgrid += dr/2
thetagrid = np.linspace(0,np.pi,nedge)
dtheta = np.diff(thetagrid)[0]
thetagrid += dtheta/2
phigrid = np.linspace(0,2*np.pi,nedge)
dphi = np.diff(phigrid)[0]
phigrid += dphi/2
rthetaphigrid = np.asarray(np.meshgrid(rgrid[:-1],thetagrid[:-1],phigrid[:-1]))
rthetaphigrid = rthetaphigrid.reshape(3,(nedge-1)**3).T
deltafactor = rthetaphigrid[:,0]**2*np.sin(rthetaphigrid[:,1])*dr*dtheta*dphi
x = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.cos(rthetaphigrid[:,2])
y = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.sin(rthetaphigrid[:,2])
z = rthetaphigrid[:,0]*np.cos(rthetaphigrid[:,1])
xyzgrid = np.dstack([x,y,z])[0]
rphizgrid = coords.rect_to_cyl(xyzgrid[:,0], xyzgrid[:,1], xyzgrid[:,2])
rphizgrid = np.dstack([rphizgrid[0],rphizgrid[1],rphizgrid[2]])[0]
denstxyz = densfunc(rphizgrid[:,0],rphizgrid[:,1],rphizgrid[:,2], params=[alpha,])
integral =  np.sum(denstxyz*deltafactor)
print('Spherical grid integration: '+str(integral))

In [ ]:
nedge = 150
rgrid = np.linspace(r_min,r_max,nedge)
dr = np.diff(rgrid)[0]
rgrid += dr/2
thetagrid = np.linspace(0,np.pi,nedge)
dtheta = np.diff(thetagrid)[0]
thetagrid += dtheta/2
phigrid = np.linspace(0,2*np.pi,nedge)
dphi = np.diff(phigrid)[0]
phigrid += dphi/2
rthetaphigrid = np.asarray(np.meshgrid(rgrid[:-1],thetagrid[:-1],phigrid[:-1]))
rthetaphigrid = rthetaphigrid.reshape(3,(nedge-1)**3).T
deltafactor = rthetaphigrid[:,0]**2*np.sin(rthetaphigrid[:,1])*dr*dtheta*dphi
x = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.cos(rthetaphigrid[:,2])
y = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.sin(rthetaphigrid[:,2])
z = rthetaphigrid[:,0]*np.cos(rthetaphigrid[:,1])
xyzgrid = np.dstack([x,y,z])[0]
rphizgrid = coords.rect_to_cyl(xyzgrid[:,0], xyzgrid[:,1], xyzgrid[:,2])
rphizgrid = np.dstack([rphizgrid[0],rphizgrid[1],rphizgrid[2]])[0]
denstxyz = densfunc(rphizgrid[:,0],rphizgrid[:,1],rphizgrid[:,2], params=[alpha,])
integral =  np.sum(denstxyz*deltafactor)
print('Spherical grid integration: '+str(integral))

In [ ]:
# Spherical grid integration
rthetaphigrid = np.mgrid[r_min:r_max:150j,0:np.pi:150j,0:2*np.pi:150j]
dr = (r_max-r_min)/149
dtheta = (np.pi-0.)/149
dphi = (2*np.pi-0.)/149
shape = np.shape(rthetaphigrid.T)
rthetaphigrid = rthetaphigrid.T.reshape(np.product(shape[:3]),shape[3])
deltafactor = rthetaphigrid[:,0]**2*np.sin(rthetaphigrid[:,1])*dr*dtheta*dphi
x = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.cos(rthetaphigrid[:,2])
y = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.sin(rthetaphigrid[:,2])
z = rthetaphigrid[:,0]*np.cos(rthetaphigrid[:,1])
xyzgrid = np.dstack([x,y,z])[0]
rphizgrid = coords.rect_to_cyl(xyzgrid[:,0], xyzgrid[:,1], xyzgrid[:,2])
rphizgrid = np.dstack([rphizgrid[0],rphizgrid[1],rphizgrid[2]])[0]
denstxyz = densfunc(rphizgrid[:,0],rphizgrid[:,1],rphizgrid[:,2], params=[alpha,])
integral =  np.sum(denstxyz*deltafactor)
print('Spherical grid integration: '+str(integral))

### Make sure that the approach outlined here matches the functionalized approach

In [ ]:
# Definitions
densfunc = pdens.spherical
# alpha, p, q, eta, theta, pa
# mcmc_labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
# mcmc_truths = [3.5,0.5,1.,0.,1.,1./6.]
# init = np.array([2.0, 0.5, 0.5, 0.5, 0.5, 0.5])
mcmc_labels = [r'$\alpha$',]
mcmc_truths = [2.5,]
init = np.array([2.0,])
ndim = len(init)
nwalkers = 100
nit = 200
ncut = 100
n_mass = 100
mass_int_type = 'spherical'
r_min,r_max = 0.5, 70.
mass_analytic = False

# Filenames
mock_number = '49'
mock_path = '../data/mock_'+mock_number+'/'
allstar_filename = mock_path+'/allstar.npy'
orbs_filename = mock_path+'/orbs.pkl'
data_omask_filename = mock_path+'/omask.npy'

# Load
with open(mock_path+'orbs.pkl','rb') as f:
    orbs_nomask = pickle.load(f)
allstar_nomask = np.load(allstar_filename)
data_mask = np.load(data_omask_filename)

# Setup data array and mask the data
orbs = orbs_nomask[data_mask]
allstar = allstar_nomask[data_mask]
mrpz = np.array([orbs.R(use_physical=True).value,
                 orbs.phi(use_physical=True).value,
                 orbs.z(use_physical=True).value]).T
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apof)
nfield = effsel.shape[0]
n_star = len(orbs)

In [ ]:
# # Maximum likelihood
# opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
#     Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
#     full_output=True)
# print(opt[0])

# #pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
# pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]

# with multiprocessing.Pool(nprocs) as pool:
#     sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
#         args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
#         pool=pool)
#     print('Generating MCMC samples...')
#     for i, result in enumerate(sampler.sample(pos, iterations=nit)):
#         if (i+1)%10 == 0: print('sampled '+str(i+1)+'/'+str(nit), end='\r')
#         continue
#     # Flatten the ensemble of walkers to a set of samples, remove ncut from each
#     # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
#     samples = sampler.get_chain(flat=True,discard=ncut)

fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()


# Determine the isochrone mass fraction factors for each field 
isofactors = np.zeros(nfield)
for i in range(nfield):
    # Calculate the full isochrone mask for the average mass calculation
    avmass_isomask = (Z2FEH(iso['Zini']) > feh_range[0]) &\
                     (Z2FEH(iso['Zini']) < feh_range[1]) &\
                     (iso['Jmag']-iso['Ksmag'] > jkmins[i]) &\
                     (iso['logg'] > logg_range[0]) &\
                     (iso['logg'] < logg_range[1]) &\
                     (iso['logAge'] >= 10) &\
                     (iso['logL'] > -9)
    avmass = piso.average_mass(iso[avmass_isomask])
    # Calculate a broader isochrone mask for the mass ratio calculation
    massratio_isomask = (Z2FEH(iso['Zini']) > feh_range[0]) &\
                        (Z2FEH(iso['Zini']) < feh_range[1]) &\
                        (iso['logAge'] >= 10) &\
                        (iso['logL'] > -9)
    massratio = piso.mass_ratio(iso[massratio_isomask], logg_range=logg_range,
                                jk_range=[jkmins[i],999.])
    isofactors[i] = avmass/massratio



# Set up grid for integration
if mass_int_type == 'spherical':
#     rthetaphigrid = np.mgrid[r_min:r_max:150j,0:np.pi:150j,0:2*np.pi:150j]
#     dr = (r_max-r_min)/149
#     dtheta = (np.pi-0.)/149
#     dphi = (2*np.pi-0.)/149
#     shape = np.shape(rthetaphigrid.T)
#     rthetaphigrid = rthetaphigrid.T.reshape(np.product(shape[:3]),shape[3])
#     deltafactor = rthetaphigrid[:,0]**2*np.sin(rthetaphigrid[:,1])*dr*dtheta*dphi
#     x = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.cos(rthetaphigrid[:,2])
#     y = rthetaphigrid[:,0]*np.sin(rthetaphigrid[:,1])*np.sin(rthetaphigrid[:,2])
#     z = rthetaphigrid[:,0]*np.cos(rthetaphigrid[:,1])
#     xyzgrid = np.dstack([x,y,z])[0]
#     rphizgrid = coords.rect_to_cyl(xyzgrid[:,0], xyzgrid[:,1], xyzgrid[:,2])
#     rphizgrid = np.dstack([rphizgrid[0],rphizgrid[1],rphizgrid[2]])[0]
    rphizgrid,deltafactor = pmass.spherical_integration_grid(r_min,r_max,500,100,100)

masses = np.zeros(n_mass)
facs = np.zeros(n_mass)

mass_analytic = False
n_star = len(orbs)

# Calculate the mass
sample_randind = np.random.choice(len(samples),n_mass,replace=False)
for ii,params in enumerate(samples[sample_randind]):
    if (ii+1)%10 == 0: print('mass samples '+str(ii+1)+'/'+str(n_mass), end='\r')
    #params = [3.5,0.5,1.,0.,1.,1./6.]
    rate = densfunc(Rgrid,phigrid,zgrid,params=params)*effsel
    sumrate = np.sum(rate.T/isofactors)
    fac = n_star/sumrate
    # fac = norm*(180./np.pi)**2 # Get rid of this normalization factor??
    if mass_analytic:
        #only for spherical power law!
        rsun = np.sqrt(ro**2+zo**2)
        alpha = params[0]
        integral = 4*np.pi*((rsun**alpha*r_max**(3-alpha))/(3-alpha)-(rsun**alpha*r_min**(3-alpha))/(3-alpha))
        masses[ii] = integral*fac
    else:
        denstxyz = densfunc(rphizgrid[:,0],rphizgrid[:,1],rphizgrid[:,2], params=params)*fac
        masses[ii] =  np.sum(denstxyz*deltafactor)
    facs[ii] = fac

print('Mass should be 5e8')
print('Median mass: '+str(np.median(masses)/1e8)+' e8')
# print('Median mass / pi: '+str(np.median(masses)/np.pi/1e8)+' e8')

In [ ]:
# Compare with the functionalized approach

samples, opt = pmass.fit_dens(densfunc, effsel, [Rgrid,phigrid,zgrid], 
    [Rdata,phidata,zdata], [3.5,], 9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)

In [ ]:
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=mcmc_truths, truth_color='Red', 
                    labels=mcmc_labels)
fig.show()

In [ ]:
masses_eq, facs_eq = pmass.mass_from_density_samples(samples=samples, 
    densfunc=densfunc, n_star=len(orbs), effsel=effsel, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

In [ ]:
np.median(masses_eq)